In [ ]:
import numpy as np
from scipy import signal

In [ ]:
def _exclude_trailing_and_leading_zeros(envelope):
    # Exclude trailing zeros (identify the last non-zero element)
    non_zero_end = np.argmax(envelope[::-1] > 0)
    envelope = envelope[:len(envelope) - non_zero_end]
    # Exclude leading zeros (find the first non-zero value)
    non_zero_start = np.argmax(envelope > 0)
    return envelope[non_zero_start:]

In [ ]:
def _dominant_freqs(amp_spectrogram, freqs, history_size=3):    
    dominant_freqs = []
    amp_range = np.max(amp_spectrogram)-np.min(amp_spectrogram)

    # Iterate over time frames
    for t in range(amp_spectrogram.shape[1]):
        # Get power spectrum for time frame
        power_spectrum = amp_spectrogram[:, t]
        peaks, _ = signal.find_peaks(power_spectrum, height=amp_range*0.05, distance=len(freqs)//50, prominence=amp_range*0.05) # 5% of amp range, 5% of freq bins, 5% of amp range

        # If no peaks, take frequency with highest amplitude
        if len(peaks) == 0:
            dominant_freq = freqs[np.argmax(power_spectrum)]
        else:
            if len(dominant_freqs) >= history_size:
                recent_avg_freq = np.mean(dominant_freqs[-history_size:])
            else:
                recent_avg_freq = np.mean(dominant_freqs) if dominant_freqs else freqs[peaks[0]]  # Fallback to the first peak

            # Choose peak closest to the previous dominant frequency
            if len(dominant_freqs) == 0:
                # for first frame
                dominant_freq = freqs[peaks[np.argmax(power_spectrum[peaks])]]
            else:
                # For subsequent frames: find peak closest to recent average frequency
                distances = np.abs(freqs[peaks] - recent_avg_freq)
                closest_peak = peaks[np.argmin(distances)]
                dominant_freq = freqs[closest_peak]

        dominant_freqs.append(dominant_freq)
    return np.asarray(dominant_freqs)

In [ ]:
def calculate_power_spectral_entropy(y, N_FFT):
        """
        Calculates the power spectral entropy as follows:
        1. Calculate Power Spectral Distribution
        2. Normalize PSD (to treat it as a measure of probability)
        3. Calculate Shannon-Wiener entropy of normalized PSD using scipy

        https://de.mathworks.com/help/signal/ref/spectralentropy.html accessed January 13th, 2025. 18:34 pm
        """
        #_, psd = signal.welch(self.y, self.sr, nperseg=N_FFT, noverlap=N_FFT//HOP_OVERLAP) # would return psd - frequency spectrum squared and scaled by sum - 
        psd = np.abs(signal.fft.fft(y, n=N_FFT)) ** 2
        psd_norm = psd / np.sum(psd)

        # TODO should I do this?
        psd_norm = _exclude_trailing_and_leading_zeros(psd_norm)
        # Ensure no zero values in normalized power distribution
        #psd_norm_nonzero = psd_norm[psd_norm > 0]
        psd_norm = np.where(psd_norm > 0, psd_norm, 1e-10)

        pse = -np.sum(psd_norm * np.log2(psd_norm))

        return pse